### 回写指令

In [ ]:
#coding:gbk

import cx_Oracle, os, csv
import pandas as pd
import numpy as np
from pandas import DataFrame
conn = cx_Oracle.connect(' ',' ',' ')

def _exec_sql(sql):
    cursor = conn.cursor()
    cursor.execute(sql)
    conn.commit()
    
def Update_SQL(new_DETLASHARE,accountid,SoF,dte,tkr):
    updatesql = " update instructs_algo set DETLASHARE={} where accountid like '%{}%{}' and dte={} and tkr='{}' and traderid=102".format(new_DETLASHARE,accountid,SoF,dte,tkr)
    _exec_sql(updatesql)
    sql = " select dte,accountid,tkr,detlashare from instructs_algo where accountid like '%{}%{}' and dte={} and tkr='{}' and traderid=102".format(accountid,SoF,dte,tkr)
    df_instr = pd.read_sql(sql,conn)
    print(df_instr.values)

def Update_SQL_BS(new_DETLASHARE_1,new_DETLASHARE_2,accountid,SoF,dte,tkr):
    de_sql = " delete from instructs_algo where accountid like '%{}%{}' and dte={} and tkr='{}' and traderid=102".format(accountid,SoF,dte,tkr)
    _exec_sql(de_sql)
    ins1_sql = " insert into instructs_algo (DTE,ACCOUNTID,TRADERID,TKR,DETLASHARE,STARTTME,ENDTME,IFUPDOWN) values ('{}','SUB_DCL5Q_903_F', '102','{}','{}','93000','113000','0')".format(dte,tkr,new_DETLASHARE_1)
    _exec_sql(ins1_sql)
    ins2_sql = " insert into instructs_algo (DTE,ACCOUNTID,TRADERID,TKR,DETLASHARE,STARTTME,ENDTME,IFUPDOWN) values ('{}','SUB_DCL5Q_903_F', '102','{}','{}','93000','113000','0')".format(dte,tkr,new_DETLASHARE_2)
    _exec_sql(ins2_sql)
    
    sql = " select * from instructs_algo where accountid like '%{}%{}' and dte={} and tkr='{}' and traderid=102".format(accountid,SoF,dte,tkr)
    df_instr = pd.read_sql(sql,conn)
    print(df_instr.values)
    
def Modify_instr(dte, accountid, SoF):
    # 实际
    sql = "select tkr,sum(qty),side from transactions_algo where dte={} and accountid like '%{}%{}' group by tkr,side".format(dte,accountid,SoF)
    df_trans = pd.read_sql(sql,conn)
    df_BS = pd.DataFrame(columns=['TKR','First'],index=[0])
    df_BS.set_index('TKR', inplace=True)
    BS_tkr = []
    for i in range(df_trans.shape[0]):
        tkr = df_trans.values[i][np.where(df_trans.columns == 'TKR')][0]
        BS = len(np.where(df_trans['TKR'] == tkr)[0])
        if BS == 1:
            if df_trans.values[i][np.where(df_trans.columns == 'SIDE')][0] == 'S':
                new_DETLASHARE = -df_trans.values[i][np.where(df_trans.columns == 'SUM(QTY)')][0]
            elif df_trans.values[i][np.where(df_trans.columns == 'SIDE')][0] == 'B':
                new_DETLASHARE = df_trans.values[i][np.where(df_trans.columns == 'SUM(QTY)')][0]
            print(df_trans.values[i])
            print(new_DETLASHARE)
            Update_SQL(new_DETLASHARE,accountid,SoF,dte,tkr)
        elif BS == 2:
            if not tkr in BS_tkr:
                BS_tkr.append(tkr)
                if df_trans.values[i][np.where(df_trans.columns == 'SIDE')][0] == 'S':
                    new_DETLASHARE_1 = -df_trans.values[i][np.where(df_trans.columns == 'SUM(QTY)')][0]
                elif df_trans.values[i][np.where(df_trans.columns == 'SIDE')][0] == 'B':
                    new_DETLASHARE_1 = df_trans.values[i][np.where(df_trans.columns == 'SUM(QTY)')][0]
                data = pd.DataFrame(columns=['First'],data = new_DETLASHARE_1,index=[tkr])
                df_BS = df_BS.append(data)
            else:
                if df_trans.values[i][np.where(df_trans.columns == 'SIDE')][0] == 'S':
                    new_DETLASHARE_2 = -df_trans.values[i][np.where(df_trans.columns == 'SUM(QTY)')][0]
                elif df_trans.values[i][np.where(df_trans.columns == 'SIDE')][0] == 'B':
                    new_DETLASHARE_2 = df_trans.values[i][np.where(df_trans.columns == 'SUM(QTY)')][0]
                new_DETLASHARE_1 = df_BS.values[np.where(df_BS.index==tkr)][0][0]
                print(tkr)
                print(new_DETLASHARE_1)
                print(new_DETLASHARE_2)
                Update_SQL_BS(new_DETLASHARE_1,new_DETLASHARE_2,accountid,SoF,dte,tkr)
            
            
    print('done!')
    
if __name__ == '__main__':
    
    #datelist = [20201103,20201104,20201106,20201109,20201116,20201117,20201125,20201130,20201119,20201113,20201216,20201218,20201221,20201223,20201224,20201225,20201207,20201208,20201209,20201211,20201215] #DCL5Q_F
    #datelist = [20201211,20201215] #DCL5Q_F剩余
    datelist = [20201026]
    #datelist = [20201102,20201104,20201105,20201110,20201217,20201221,20201223,20201211,20201214] #ZZ1H
    #datelist = [20201102]  #ZZ1H剩余
    for dte in datelist:
        dte = str(dte)
        accountid = 'DCL5Q'  # 记得修改 Update_SQL_BS 中的insert子产品名字！！
        SoF = 'F'
    
        Modify_instr(dte, accountid, SoF)